In [6]:
from ROOT import *
import numpy as np
from prettytable import PrettyTable
import sys
import uncertainties as unc # propagate uncertainties
from uncertainties import unumpy as unp # array operations for type ufloat
import os
    
var_array = ["bdt","eta", "c1", "width","ntrk"]
var_title_array = ["BDT score","#eta","C_{1}^{#beta=0.2}","W_{trk}","N_{trk}"] 
mc = "pythia"   #by setting it as "SF" or "MC", it will automatically making scale factor plots or MC closure plots

eta_bin = [0.5]



fdijetMC = TFile("/eos/user/r/rqian/dijet-mono-result/pythia-small-eta/dijet_pythia_small_eta_test.root")

bins = np.array([500.,600.,800.,1000.,1200.,1500.,2000.])
bin = np.array([500,600,800,1000,1200,1500,2000])

path = mc+"_mc"
isExists=os.path.exists(mc+"_mc")
if not isExists:
        os.makedirs(mc+"_mc")        

#convert histogram and error into unp.uarray
#if sample is pyroot input, GetBinError returns correct result
def unc_array(hist):
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = hist.GetBinError(j)
    result = unp.uarray(value,error)
    return(result)

#convert histogram and error into unp.uarray
#if sample is uproot input, err is the sumw2 of the corresponding histogram
def unc_array_err(hist,err):
    
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = np.sqrt(err.GetBinContent(j))
    result = unp.uarray(value,error)
    return(result)


def myText(x,y,text, color = 1):
    l = TLatex()
    l.SetTextSize(0.025)
    l.SetNDC()
    l.SetTextColor(color)
    l.DrawLatex(x,y,text)
    pass

def error_check(hist):
    for i in range(1,hist.GetNbinsX()+1):
        if hist.GetBinContent(i)>1:
            hist.SetBinContent(i,0)
    return(hist)



# convert uncertainty array into histogram
def set_hist(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))
        hist.SetBinContent(i,unp.nominal_values(unc[i-1]))
        
def set_hist_error(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))

#inclusive fraction calculation
def fraction(lower_quark,lower_gluon,higher_quark,higher_gluon):
    ToT_Fq2 = 0.
    ToT_Fg2 = 0.
    ToT_Cq2 =0.
    ToT_Cg2 = 0.

    for j in range(1,lower_quark.GetNbinsX()+1):
        ToT_Fq2+=higher_quark.GetBinContent(j)
        ToT_Cq2+=lower_quark.GetBinContent(j)
        ToT_Fg2+=higher_gluon.GetBinContent(j)
        ToT_Cg2+=lower_gluon.GetBinContent(j)

    # calculate the fraction of each sample 
    if ((ToT_Fg2+ToT_Fq2) != 0):
        fg=ToT_Fg2/(ToT_Fg2+ToT_Fq2)
        cg=ToT_Cg2/(ToT_Cq2+ToT_Cg2)

    fq=1.-fg
    cq=1.-cg
    return(fg,cg,fq,cq)


    
    

    
def ReadingDijetDataEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_data = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var)
    lower_data = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var)

    higher_data.SetName(pt+"_LeadingJet_Forward_Data_"+str(index)+var+name)
    lower_data.SetName(pt+"_LeadingJet_Central_Data_"+str(index)+var+name)
    
    higher_data_2 = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var)
    lower_data_2 = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var)
    
    higher_data_2.SetName(pt+"_SubJet_Forward_Data_"+str(index)+var+name)
    lower_data_2.SetName(pt+"_SubJet_Central_Data_"+str(index)+var+name)
    
    higher_data.Add(higher_data_2)
    lower_data.Add(lower_data_2)
    

    higher_data_err = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_err = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_err.SetName(pt+"_LeadingJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_err.SetName(pt+"_LeadingJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_2_err = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_2_err = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_2_err.SetName(pt+"_SubJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_2_err.SetName(pt+"_SubJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_err.Add(higher_data_2_err)
    lower_data_err.Add(lower_data_2_err)
    
    for i in range(1,higher_data.GetNbinsX()+1):
        higher_data.SetBinError(i,np.sqrt(higher_data_err.GetBinContent(i)))
        lower_data.SetBinError(i,np.sqrt(lower_data_err.GetBinContent(i)))
        
    histo_dic["hd"] =  higher_data
    histo_dic["ld"] =  lower_data

    return(histo_dic)        


def ReadingDijetMCEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_quark = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var)


    higher_quark.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+var+name)
    lower_quark.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+var+name)
    higher_gluon.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+var+name)
    

    higher_quark_2 = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark_2 = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon_2 = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon_2 = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark_2 = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark_2 = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark_2 = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark_2 = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var)

    higher_quark_2.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+var+name)
    lower_quark_2.SetName(pt+"_SubJet_Central_Quark_"+str(index)+var+name)
    higher_gluon_2.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon_2.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark_2.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark_2.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark_2.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark_2.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+var+name)

    higher_quark.Add(higher_quark_2)
    higher_gluon.Add(higher_gluon_2)
    higher_b_quark.Add(higher_b_quark_2)
    higher_c_quark.Add(higher_c_quark_2)

    lower_quark.Add(lower_quark_2)
    lower_gluon.Add(lower_gluon_2)
    lower_b_quark.Add(lower_b_quark_2)
    lower_c_quark.Add(lower_c_quark_2)
 
            
    higher_b_quark.Add(higher_c_quark)
    lower_b_quark.Add(lower_c_quark)

    higher_quark_err = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_err = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_err = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_err = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_err = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_err = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_err = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_err = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err")



    
    higher_quark_err.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_err.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_err.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_err.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_err.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_err.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_err.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_err.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_2_err = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_2_err = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_2_err = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_2_err = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_2_err = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_2_err = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_2_err = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_2_err = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err")
    
    higher_quark_2_err.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_2_err.SetName(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_2_err.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_2_err.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_2_err.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_2_err.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_2_err.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_2_err.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_err.Add(higher_quark_2_err)
    higher_gluon_err.Add(higher_gluon_2_err)
    higher_b_quark_err.Add(higher_b_quark_2_err)
    higher_c_quark_err.Add(higher_c_quark_2_err)

    lower_quark_err.Add(lower_quark_2_err)
    lower_gluon_err.Add(lower_gluon_2_err)
    lower_b_quark_err.Add(lower_b_quark_2_err)
    lower_c_quark_err.Add(lower_c_quark_2_err)
 
            
    higher_b_quark_err.Add(higher_c_quark_err)
    lower_b_quark_err.Add(lower_c_quark_err)
    
    for i in range(1,higher_quark.GetNbinsX()+1):
        higher_quark.SetBinError(i,np.sqrt(higher_quark_err.GetBinContent(i)))
        higher_gluon.SetBinError(i,np.sqrt(higher_gluon_err.GetBinContent(i)))
        higher_b_quark.SetBinError(i,np.sqrt(higher_b_quark_err.GetBinContent(i)))
        lower_quark.SetBinError(i,np.sqrt(lower_quark_err.GetBinContent(i)))
        lower_gluon.SetBinError(i,np.sqrt(lower_gluon_err.GetBinContent(i)))
        lower_b_quark.SetBinError(i,np.sqrt(lower_b_quark_err.GetBinContent(i)))

    
    histo_dic["hq"] =  higher_quark
    histo_dic["hg"] =  higher_gluon
    histo_dic["ho"] =  higher_b_quark
    
    histo_dic["hmc"] =  higher_quark.Clone()
    histo_dic["hmc"].SetName(pt+"higher_all_jets"+var+"_"+name)
    histo_dic["hmc"].Add(higher_gluon)
    histo_dic["hmc"].Add(higher_b_quark)
    
    histo_dic["lq"] =  lower_quark
    histo_dic["lg"] =  lower_gluon
    histo_dic["lo"] =  lower_b_quark
    
    histo_dic["lmc"] =  lower_quark.Clone()
    histo_dic["lmc"].SetName(pt+"lower_all_jets"+var+"_"+name)
    histo_dic["lmc"].Add(lower_gluon)
    histo_dic["lmc"].Add(lower_b_quark)
    
    return(histo_dic)





gStyle.SetOptStat(0)

for k in range(0,6):  #for only dijet event, start frTom jet pT>500 GeV

     for var_index,var in enumerate(var_array):
    
            min = bin[k]
            max = bin[k+1]
            for e in range(0,len(eta_bin)):
        
                    print(var,str(eta_bin[e]))
                    
                    nominal_jets =  ReadingDijetMCEta(fdijetMC,str(min),var,str(eta_bin[e]),str(eta_bin[e]))
                
                            
                    higher_quark = nominal_jets["hq"]
                    lower_quark = nominal_jets["lq"]
                    higher_gluon = nominal_jets["hg"]
                    lower_gluon = nominal_jets["lg"]
                    higher_other = nominal_jets["ho"]
                    lower_other = nominal_jets["lo"]        
                    higher_mc = nominal_jets["hmc"]
                    lower_mc = nominal_jets["lmc"]        

                    
                    higher_quark_unc = unc_array(higher_quark)
                    higher_gluon_unc = unc_array(higher_gluon)
                    lower_quark_unc = unc_array(lower_quark)
                    lower_gluon_unc = unc_array(lower_gluon)
                    higher_mc_unc = unc_array(higher_mc)
                    lower_mc_unc = unc_array(lower_mc)
        
                    
                    ToT_Fq2_unc = higher_quark_unc.sum()
                    ToT_Fg2_unc = higher_gluon_unc.sum()
            
                    ToT_Cq2_unc = lower_quark_unc.sum()
                    ToT_Cg2_unc = lower_gluon_unc.sum()
            
                    # calculate the fraction of forward(higher) / central(lower) quark or gluon jet
                    fg_unc=ToT_Fg2_unc/(ToT_Fg2_unc+ToT_Fq2_unc)
                    cg_unc=ToT_Cg2_unc/(ToT_Cq2_unc+ToT_Cg2_unc)
                    fq_unc=1.-fg_unc
                    cq_unc=1.-cg_unc
            
                    factor_quark_unc = lower_quark_unc
                    factor_gluon_unc = lower_gluon_unc
                    
                    
                    higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
                    higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
                    lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
                    lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()
        
                    higher_mc_unc = higher_mc_unc/higher_mc_unc.sum()
                    lower_mc_unc = lower_mc_unc/lower_mc_unc.sum()        

        
                        
                    higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
                    higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
                    lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
                    lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()
        
                    higher_mc_unc = higher_mc_unc/higher_mc_unc.sum()
                    lower_mc_unc = lower_mc_unc/lower_mc_unc.sum()   
                    
                    higher = higher_mc.Clone()
                    lower = lower_mc.Clone()
            
                    higher_unc = higher_mc_unc
                    lower_unc  = lower_mc_unc
            
                    
        
                    # kinematics
                    set_hist(higher_quark,higher_quark_unc)
                    set_hist(higher_gluon,higher_gluon_unc)
                    set_hist(lower_quark,lower_quark_unc)
                    set_hist(lower_gluon,lower_gluon_unc)   
                    
                    
                    higher_quark_hist = higher_quark.Clone()
                    higher_gluon_hist = higher_quark.Clone() 
                    lower_quark_hist = higher_quark.Clone()
                    lower_gluon_hist = higher_quark.Clone()
        
                    for i in range(1,higher_quark.GetNbinsX()+1):
                            if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                    factor_gluon_unc[i-1] = higher_gluon_unc[i-1]/lower_gluon_unc[i-1]
                                    factor_quark_unc[i-1] = higher_quark_unc[i-1]/lower_quark_unc[i-1]
                            else:
                                    factor_gluon_unc[i-1] = unc.ufloat(1, 0)
                                    factor_quark_unc[i-1] = unc.ufloat(1, 0)                    
                    for i in range(1,higher_quark.GetNbinsX()+1):
                        higher_quark_hist.SetBinContent(i,higher_quark_unc[i-1].nominal_value)
                        higher_quark_hist.SetBinError(i,higher_quark_unc[i-1].std_dev)
                        lower_quark_hist.SetBinContent(i,lower_quark_unc[i-1].nominal_value)
                        lower_quark_hist.SetBinError(i,lower_quark_unc[i-1].std_dev)
                        higher_gluon_hist.SetBinContent(i,higher_gluon_unc[i-1].nominal_value)
                        higher_gluon_hist.SetBinError(i,higher_gluon_unc[i-1].std_dev)
                        lower_gluon_hist.SetBinContent(i,lower_gluon_unc[i-1].nominal_value)
                        lower_gluon_hist.SetBinError(i,lower_gluon_unc[i-1].std_dev)
                        
                    factor_quark_hist = higher_quark.Clone()
                    factor_gluon_hist = higher_gluon.Clone()
                    for i in range(1,higher_quark.GetNbinsX()+1):
        
                            factor_quark_hist.SetBinContent(i,factor_quark_unc[i-1].nominal_value)
                            factor_quark_hist.SetBinError(i,factor_quark_unc[i-1].std_dev)
                            factor_gluon_hist.SetBinContent(i,factor_gluon_unc[i-1].nominal_value)
                            factor_gluon_hist.SetBinError(i,factor_gluon_unc[i-1].std_dev)
                    
                           
                    
                    c5 = TCanvas("","",500,500)
                    c5.Divide(2,1)
        
                    top = c5.cd(1)
                    top.SetPad(0.0,0.0,1.0,1.0)
                    top.SetFillColor(0)
                    top.SetBorderMode(0)
                    top.SetBorderSize(2)
                    top.SetTickx(1)
                    top.SetTicky(1)
                    top.SetLeftMargin(0.14)
                    top.SetRightMargin(0.055)
                    top.SetBottomMargin(0.3)#0.25
                    top.SetFrameBorderMode(0)
                    #top.SetLogy(1)
                    top.cd()
                    higher_quark_hist.SetMarkerColor(4)
                    higher_quark_hist.SetLineColor(4)
                    higher_quark_hist.SetMarkerSize(0.5)
                    higher_quark_hist.SetLineStyle(1)
                    higher_gluon_hist.SetMarkerColor(2)
                    higher_gluon_hist.SetLineColor(2)
                    higher_gluon_hist.SetMarkerSize(0.5)
                    higher_gluon_hist.SetLineStyle(1)         
                    lower_quark_hist.SetMarkerColor(4)
                    lower_quark_hist.SetLineColor(4)
                    lower_quark_hist.SetMarkerSize(0.5)
                    lower_quark_hist.SetLineStyle(2)
                    lower_gluon_hist.SetMarkerColor(2)
                    lower_gluon_hist.SetLineColor(2)
                    lower_gluon_hist.SetMarkerSize(0.5)
                    lower_gluon_hist.SetLineStyle(2)           
                    bot = c5.cd(2)
                    bot.SetPad(0.0,0.0,1.0,0.3)
                    bot.SetFillColor(0)
                    bot.SetBorderMode(0)
                    bot.SetBorderSize(2)
                    bot.SetTickx(1)
                    bot.SetTicky(1)
                    bot.SetLeftMargin(0.14)
                    bot.SetRightMargin(0.055)
                    bot.SetTopMargin(0.045)
                    bot.SetBottomMargin(0.4)
                    bot.SetFrameBorderMode(0)   
                    factor_quark_hist.SetMarkerColor(4)
                    factor_quark_hist.SetLineColor(4)
                    factor_quark_hist.SetMarkerSize(0.5)
                    factor_quark_hist.SetLineStyle(1)
                    factor_gluon_hist.SetMarkerColor(2)
                    factor_gluon_hist.SetLineColor(2)
                    factor_gluon_hist.SetMarkerSize(0.5)
                    factor_gluon_hist.SetLineStyle(1)   
                    #leg.AddEntry(gluon_data,"Extracted gluon (data)","p")
                    factor_quark_hist.SetMinimum(0)
                    factor_quark_hist.SetMaximum(2)
                    factor_quark_hist.GetYaxis().SetTitle("Forward/Central")        
                    factor_quark_hist.GetYaxis().SetRangeUser(0.7,1.3)
                    factor_quark_hist.GetXaxis().SetTitleOffset(1)
                    factor_quark_hist.GetXaxis().SetTitleSize(0.11)
                    factor_quark_hist.GetXaxis().SetLabelSize(0.1)
                    factor_quark_hist.GetXaxis().SetLabelOffset(0.03)
                    factor_quark_hist.GetYaxis().SetTitleSize(0.1)
                    factor_quark_hist.GetYaxis().SetTitleOffset(0.5)
                    factor_quark_hist.GetYaxis().SetLabelOffset(0.01)
                    factor_quark_hist.GetYaxis().SetLabelSize(0.1)
                    factor_quark_hist.GetYaxis().SetLabelOffset(0.02)
                    top.cd()
                    rmax = higher_gluon_hist.GetMaximum()*1.5
                    higher_quark_hist.GetYaxis().SetTitle("Normalized to unity")        
                    higher_quark_hist.SetMaximum(rmax)
                    higher_quark_hist.Draw("HIST")
                    higher_gluon_hist.Draw("HIST same")
                    lower_quark_hist.Draw("HIST same")
                    lower_gluon_hist.Draw("HIST same")

                    factor_quark_hist.GetXaxis().SetTitle(var_title_array[var_index])
                    leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  

        
                    leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  
                    leg.AddEntry(higher_quark_hist,"forward quark","l")
                    leg.AddEntry(lower_quark_hist,"central quark","l")     
                    leg.AddEntry(higher_gluon_hist,"forward gluon","l")
                    leg.AddEntry(lower_gluon_hist,"central gluon","l")  
                    leg.SetTextFont(42)
                    leg.SetFillColor(0)
                    leg.SetBorderSize(0)
                    leg.SetFillStyle(0)
                    leg.SetNColumns(1)
                    leg.Draw()        
                    myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
                    myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
                    myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
                    myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
                    myText(0.18,0.70,"0<|\eta_{C}|<0.5,0<|\eta_{F}|<0.5")
                    bot.cd()
                    factor_quark_hist.Draw("HIST e")
                    factor_gluon_hist.Draw("HIST e same")
        
                    c5.Print(var+"_"+str(min)+"_"+mc+"_"+var+"_"+str(eta_bin[e])+"_distribution.png")           

bdt 0.5
eta 0.5
c1 0.5
width 0.5
ntrk 0.5
bdt 0.5
eta 0.5
c1 0.5
width 0.5
ntrk 0.5
bdt 0.5
eta 0.5
c1 0.5
width 0.5
ntrk 0.5
bdt 0.5
eta 0.5
c1 0.5
width 0.5
ntrk 0.5
bdt 0.5
eta 0.5
c1 0.5
width 0.5
ntrk 0.5
bdt 0.5
eta 0.5
c1 0.5
width 0.5
ntrk 0.5


Info in <TCanvas::Print>: png file bdt_500_pythia_bdt_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file eta_500_pythia_eta_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file c1_500_pythia_c1_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file width_500_pythia_width_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file ntrk_500_pythia_ntrk_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file bdt_600_pythia_bdt_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file eta_600_pythia_eta_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file c1_600_pythia_c1_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file width_600_pythia_width_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file ntrk_600_pythia_ntrk_0.5_distribution.png has been created
Info in <TCanvas::Print>: png file bdt_800_pythia_bdt_0.5_distribution.p

In [7]:
#0-0.5 0.5-1

from ROOT import *
import numpy as np
from prettytable import PrettyTable
import sys
import uncertainties as unc # propagate uncertainties
from uncertainties import unumpy as unp # array operations for type ufloat
import os
    
var_array = ["bdt","eta", "c1", "width","ntrk"]
var_title_array = ["BDT","#eta","C_{1}^{#beta=0.2}","W_{trk}","N_{trk}"] 
mc = "pythia"   #by setting it as "SF" or "MC", it will automatically making scale factor plots or MC closure plots

 
    
eta_bin = ["0-0.5_0.5-1"]



fdijetMC = TFile("/eos/user/r/rqian/dijet-mono-result/pythia-eta/testdijet_pythia_etas.root")

bins = np.array([500.,600.,800.,1000.,1200.,1500.,2000.])
bin = np.array([500,600,800,1000,1200,1500,2000])

path = mc+"_mc"
isExists=os.path.exists(mc+"_mc")
if not isExists:
        os.makedirs(mc+"_mc")        

#convert histogram and error into unp.uarray
#if sample is pyroot input, GetBinError returns correct result
def unc_array(hist):
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = hist.GetBinError(j)
    result = unp.uarray(value,error)
    return(result)

#convert histogram and error into unp.uarray
#if sample is uproot input, err is the sumw2 of the corresponding histogram
def unc_array_err(hist,err):
    
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = np.sqrt(err.GetBinContent(j))
    result = unp.uarray(value,error)
    return(result)


def myText(x,y,text, color = 1):
    l = TLatex()
    l.SetTextSize(0.025)
    l.SetNDC()
    l.SetTextColor(color)
    l.DrawLatex(x,y,text)
    pass

def error_check(hist):
    for i in range(1,hist.GetNbinsX()+1):
        if hist.GetBinContent(i)>1:
            hist.SetBinContent(i,0)
    return(hist)



# convert uncertainty array into histogram
def set_hist(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))
        hist.SetBinContent(i,unp.nominal_values(unc[i-1]))
        
def set_hist_error(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))

#inclusive fraction calculation
def fraction(lower_quark,lower_gluon,higher_quark,higher_gluon):
    ToT_Fq2 = 0.
    ToT_Fg2 = 0.
    ToT_Cq2 =0.
    ToT_Cg2 = 0.

    for j in range(1,lower_quark.GetNbinsX()+1):
        ToT_Fq2+=higher_quark.GetBinContent(j)
        ToT_Cq2+=lower_quark.GetBinContent(j)
        ToT_Fg2+=higher_gluon.GetBinContent(j)
        ToT_Cg2+=lower_gluon.GetBinContent(j)

    # calculate the fraction of each sample 
    if ((ToT_Fg2+ToT_Fq2) != 0):
        fg=ToT_Fg2/(ToT_Fg2+ToT_Fq2)
        cg=ToT_Cg2/(ToT_Cq2+ToT_Cg2)

    fq=1.-fg
    cq=1.-cg
    return(fg,cg,fq,cq)

def mc_matrixmethod(lower_quark_input,lower_gluon_input,higher_quark_input,higher_gluon_input,fg,cg,fq,cq,higher_mc_input,lower_mc_input):
        lower_quark = lower_quark_input.Clone()
        lower_gluon = lower_gluon_input.Clone()
        higher_quark = higher_quark_input.Clone()
        higher_gluon = higher_gluon_input.Clone()
        lower_mc = lower_mc_input.Clone()
        higher_mc = higher_mc_input.Clone()
        if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_quark)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_quark)
                                lower_mc.SetBinContent(i,lower_mc.GetBinContent(i)*factor_quark)


            
        if (doreweight=="Gluon"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_gluon)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_gluon)
                                lower_mc.SetBinContent(i,lower_mc.GetBinContent(i)*factor_gluon)

        
        if (lower_quark.Integral() != 0):
            lower_quark.Scale(1./lower_quark.Integral())
        if(lower_gluon.Integral() != 0):
            lower_gluon.Scale(1./lower_gluon.Integral())
        if(higher_quark.Integral() != 0):
            higher_quark.Scale(1./higher_quark.Integral())
        if(higher_gluon.Integral() != 0):
            higher_gluon.Scale(1./higher_gluon.Integral()) 
        if(lower_mc.Integral() != 0):
                lower_mc.Scale(1./lower_mc.Integral())
        if(higher_mc.Integral() != 0):
                higher_mc.Scale(1./higher_mc.Integral())

        higher = higher_mc.Clone()
        lower = lower_mc.Clone()          


        #Now, let's solve.
        quark_extracted = higher_quark.Clone()
        gluon_extracted = higher_quark.Clone()

        #Matrix method here
        for i in range(1,higher.GetNbinsX()+1):
                F = higher.GetBinContent(i)
                C = lower.GetBinContent(i)
                if((cg*fq-fg*cq) != 0 ):
                        Q = -(C*fg-F*cg)/(cg*fq-fg*cq)
                        G = (C*fq-F*cq)/(cg*fq-fg*cq)
                        quark_extracted.SetBinContent(i,Q)
                        gluon_extracted.SetBinContent(i,G)
                        #print "   ",i,G,higher_gluon.GetBinContent(i),lower_gluon.GetBinContent(i)
                pass
            
        return(quark_extracted,gluon_extracted)


# do matrix on data for given normalized distribution
def data_matrixmethod(lower_quark_input,lower_gluon_input,higher_quark_input,higher_gluon_input,higher_input,lower_input,fg,cg,fq,cq):
        lower_quark = lower_quark_input.Clone()
        lower_gluon = lower_gluon_input.Clone()
        higher_quark = higher_quark_input.Clone()
        higher_gluon = higher_gluon_input.Clone()
        lower = lower_input.Clone()
        higher = higher_input.Clone()
        if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_quark)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_quark)
                                lower.SetBinContent(i,lower.GetBinContent(i)*factor_quark)    
        if (doreweight=="Gluon"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_gluon)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_gluon)
                                lower.SetBinContent(i,lower.GetBinContent(i)*factor_gluon)                                
        if (lower_quark.Integral() != 0):
            lower_quark.Scale(1./lower_quark.Integral())
        if(lower_gluon.Integral() != 0):
            lower_gluon.Scale(1./lower_gluon.Integral())
        if(higher_quark.Integral() != 0):
            higher_quark.Scale(1./higher_quark.Integral())
        if(higher_gluon.Integral() != 0):
            higher_gluon.Scale(1./higher_gluon.Integral()) 
        if(lower.Integral() != 0):
            lower.Scale(1./lower.Integral()) 
        if(higher.Integral() != 0):
            higher.Scale(1./higher.Integral())         
        #Now, let's solve.
        quark_extracted = higher_quark.Clone()
        gluon_extracted = higher_quark.Clone()
        #if min == 500:
        #    for j in range(1,61):
        #        print("testflag3:",j,higher.GetBinContent(j),lower.GetBinContent(j))

        #Matrix method here
        for i in range(1,higher.GetNbinsX()+1):
                F = higher.GetBinContent(i)
                C = lower.GetBinContent(i)
                if((cg*fq-fg*cq) != 0 ):
                        Q = -(C*fg-F*cg)/(cg*fq-fg*cq)
                        G = (C*fq-F*cq)/(cg*fq-fg*cq)
                        quark_extracted.SetBinContent(i,Q)
                        gluon_extracted.SetBinContent(i,G)
                        #print "   ",i,G,higher_gluon.GetBinContent(i),lower_gluon.GetBinContent(i)
                pass
            
        return(quark_extracted,gluon_extracted)
    
    

    
def ReadingDijetDataEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_data = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var)
    lower_data = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var)

    higher_data.SetName(pt+"_LeadingJet_Forward_Data_"+str(index)+var+name)
    lower_data.SetName(pt+"_LeadingJet_Central_Data_"+str(index)+var+name)
    
    higher_data_2 = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var)
    lower_data_2 = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var)
    
    higher_data_2.SetName(pt+"_SubJet_Forward_Data_"+str(index)+var+name)
    lower_data_2.SetName(pt+"_SubJet_Central_Data_"+str(index)+var+name)
    
    higher_data.Add(higher_data_2)
    lower_data.Add(lower_data_2)
    

    higher_data_err = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_err = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_err.SetName(pt+"_LeadingJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_err.SetName(pt+"_LeadingJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_2_err = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_2_err = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_2_err.SetName(pt+"_SubJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_2_err.SetName(pt+"_SubJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_err.Add(higher_data_2_err)
    lower_data_err.Add(lower_data_2_err)
    
    for i in range(1,higher_data.GetNbinsX()+1):
        higher_data.SetBinError(i,np.sqrt(higher_data_err.GetBinContent(i)))
        lower_data.SetBinError(i,np.sqrt(lower_data_err.GetBinContent(i)))
        
    histo_dic["hd"] =  higher_data
    histo_dic["ld"] =  lower_data

    return(histo_dic)        


def ReadingDijetMCEta(file0,pt,var,eta,name):
    histo_dic = {}
    higher_quark = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Forward_Quark_"+var)
    lower_quark = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Central_Quark_"+var)
    higher_gluon = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Forward_Gluon_"+var)
    lower_gluon = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Central_Gluon_"+var)
    higher_b_quark = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Forward_B_Quark_"+var)
    lower_b_quark = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Central_B_Quark_"+var)
    higher_c_quark = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Forward_C_Quark_"+var)
    lower_c_quark = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Central_C_Quark_"+var)


    higher_quark_2 = file0.Get(str(eta)+"_"+pt+"_SubJet_Forward_Quark_"+var)
    lower_quark_2 = file0.Get(str(eta)+"_"+pt+"_SubJet_Central_Quark_"+var)
    higher_gluon_2 = file0.Get(str(eta)+"_"+pt+"_SubJet_Forward_Gluon_"+var)
    lower_gluon_2 = file0.Get(str(eta)+"_"+pt+"_SubJet_Central_Gluon_"+var)
    higher_b_quark_2 = file0.Get(str(eta)+"_"+pt+"_SubJet_Forward_B_Quark_"+var)
    lower_b_quark_2 = file0.Get(str(eta)+"_"+pt+"_SubJet_Central_B_Quark_"+var)
    higher_c_quark_2 = file0.Get(str(eta)+"_"+pt+"_SubJet_Forward_C_Quark_"+var)
    lower_c_quark_2 = file0.Get(str(eta)+"_"+pt+"_SubJet_Central_C_Quark_"+var)

    
    higher_quark.SetName(str(eta)+"_"+pt+"_LeadingJet_Forward_Quark_"+var+name)
    lower_quark.SetName(str(eta)+"_"+pt+"_LeadingJet_Central_Quark_"+var+name)
    higher_gluon.SetName(str(eta)+"_"+pt+"_LeadingJet_Forward_Gluon_"+var+name)
    lower_gluon.SetName(str(eta)+"_"+pt+"_LeadingJet_Central_Gluon_"+var+name)
    higher_b_quark.SetName(str(eta)+"_"+pt+"_LeadingJet_Forward_B_Quark_"+var+name)
    lower_b_quark.SetName(str(eta)+"_"+pt+"_LeadingJet_Central_B_Quark_"+var+name)
    higher_c_quark.SetName(str(eta)+"_"+pt+"_LeadingJet_Forward_C_Quark_"+var+name)
    lower_c_quark.SetName(str(eta)+"_"+pt+"_LeadingJet_Central_C_Quark_"+var+name)


    higher_quark_2.SetName(str(eta)+"_"+pt+"_SubJet_Forward_Quark_"+var+name)
    lower_quark_2.SetName(str(eta)+"_"+pt+"_SubJet_Central_Quark_"+var+name)
    higher_gluon_2.SetName(str(eta)+"_"+pt+"_SubJet_Forward_Gluon_"+var+name)
    lower_gluon_2.SetName(str(eta)+"_"+pt+"_SubJet_Central_Gluon_"+var+name)
    higher_b_quark_2.SetName(str(eta)+"_"+pt+"_SubJet_Forward_B_Quark_"+var+name)
    lower_b_quark_2.SetName(str(eta)+"_"+pt+"_SubJet_Central_B_Quark_"+var+name)
    higher_c_quark_2.SetName(str(eta)+"_"+pt+"_SubJet_Forward_C_Quark_"+var+name)
    lower_c_quark_2.SetName(str(eta)+"_"+pt+"_SubJet_Central_C_Quark_"+var+name)

    higher_quark.Add(higher_quark_2)
    higher_gluon.Add(higher_gluon_2)
    higher_b_quark.Add(higher_b_quark_2)
    higher_c_quark.Add(higher_c_quark_2)

    lower_quark.Add(lower_quark_2)
    lower_gluon.Add(lower_gluon_2)
    lower_b_quark.Add(lower_b_quark_2)
    lower_c_quark.Add(lower_c_quark_2)
 
            
    higher_b_quark.Add(higher_c_quark)
    lower_b_quark.Add(lower_c_quark)

    higher_quark_err = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Forward_Quark_"+var+"_err")
    lower_quark_err = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Central_Quark_"+var+"_err")
    higher_gluon_err = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Forward_Gluon_"+var+"_err")
    lower_gluon_err = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Central_Gluon_"+var+"_err")
    higher_b_quark_err = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Forward_B_Quark_"+var+"_err")
    lower_b_quark_err = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Central_B_Quark_"+var+"_err")
    higher_c_quark_err = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Forward_C_Quark_"+var+"_err")
    lower_c_quark_err = file0.Get(str(eta)+"_"+pt+"_LeadingJet_Central_C_Quark_"+var+"_err")


    higher_quark_2_err = file0.Get(str(eta)+"_"+pt+"_SubJet_Forward_Quark_"+var+"_err")
    lower_quark_2_err = file0.Get(str(eta)+"_"+pt+"_SubJet_Central_Quark_"+var+"_err")
    higher_gluon_2_err = file0.Get(str(eta)+"_"+pt+"_SubJet_Forward_Gluon_"+var+"_err")
    lower_gluon_2_err = file0.Get(str(eta)+"_"+pt+"_SubJet_Central_Gluon_"+var+"_err")
    higher_b_quark_2_err = file0.Get(str(eta)+"_"+pt+"_SubJet_Forward_B_Quark_"+var+"_err")
    lower_b_quark_2_err = file0.Get(str(eta)+"_"+pt+"_SubJet_Central_B_Quark_"+var+"_err")
    higher_c_quark_2_err = file0.Get(str(eta)+"_"+pt+"_SubJet_Forward_C_Quark_"+var+"_err")
    lower_c_quark_2_err = file0.Get(str(eta)+"_"+pt+"_SubJet_Central_C_Quark_"+var+"_err")

    
    higher_quark_err.SetName(str(eta)+"_"+pt+"_LeadingJet_Forward_Quark_"+var+"_err"+name)
    lower_quark_err.SetName(str(eta)+"_"+pt+"_LeadingJet_Central_Quark_"+var+"_err"+name)
    higher_gluon_err.SetName(str(eta)+"_"+pt+"_LeadingJet_Forward_Gluon_"+var+"_err"+name)
    lower_gluon_err.SetName(str(eta)+"_"+pt+"_LeadingJet_Central_Gluon_"+var+"_err"+name)
    higher_b_quark_err.SetName(str(eta)+"_"+pt+"_LeadingJet_Forward_B_Quark_"+var+"_err"+name)
    lower_b_quark_err.SetName(str(eta)+"_"+pt+"_LeadingJet_Central_B_Quark_"+var+"_err"+name)
    higher_c_quark_err.SetName(str(eta)+"_"+pt+"_LeadingJet_Forward_C_Quark_"+var+"_err"+name)
    lower_c_quark_err.SetName(str(eta)+"_"+pt+"_LeadingJet_Central_C_Quark_"+var+"_err"+name)


    higher_quark_2_err.SetName(str(eta)+"_"+pt+"_SubJet_Forward_Quark_"+var+"_err"+name)
    lower_quark_2_err.SetName(str(eta)+"_"+pt+"_SubJet_Central_Quark_"+var+"_err"+name)
    higher_gluon_2_err.SetName(str(eta)+"_"+pt+"_SubJet_Forward_Gluon_"+var+"_err"+name)
    lower_gluon_2_err.SetName(str(eta)+"_"+pt+"_SubJet_Central_Gluon_"+var+"_err"+name)
    higher_b_quark_2_err.SetName(str(eta)+"_"+pt+"_SubJet_Forward_B_Quark_"+var+"_err"+name)
    lower_b_quark_2_err.SetName(str(eta)+"_"+pt+"_SubJet_Central_B_Quark_"+var+"_err"+name)
    higher_c_quark_2_err.SetName(str(eta)+"_"+pt+"_SubJet_Forward_C_Quark_"+var+"_err"+name)
    lower_c_quark_2_err.SetName(str(eta)+"_"+pt+"_SubJet_Central_C_Quark_"+var+"_err"+name)

    higher_quark_err.Add(higher_quark_2_err)
    higher_gluon_err.Add(higher_gluon_2_err)
    higher_b_quark_err.Add(higher_b_quark_2_err)
    higher_c_quark_err.Add(higher_c_quark_2_err)

    lower_quark_err.Add(lower_quark_2_err)
    lower_gluon_err.Add(lower_gluon_2_err)
    lower_b_quark_err.Add(lower_b_quark_2_err)
    lower_c_quark_err.Add(lower_c_quark_2_err)
 
            
    higher_b_quark_err.Add(higher_c_quark_err)
    lower_b_quark_err.Add(lower_c_quark_err)
    
    for i in range(1,higher_quark.GetNbinsX()+1):
        higher_quark.SetBinError(i,np.sqrt(higher_quark_err.GetBinContent(i)))
        higher_gluon.SetBinError(i,np.sqrt(higher_gluon_err.GetBinContent(i)))
        higher_b_quark.SetBinError(i,np.sqrt(higher_b_quark_err.GetBinContent(i)))
        lower_quark.SetBinError(i,np.sqrt(lower_quark_err.GetBinContent(i)))
        lower_gluon.SetBinError(i,np.sqrt(lower_gluon_err.GetBinContent(i)))
        lower_b_quark.SetBinError(i,np.sqrt(lower_b_quark_err.GetBinContent(i)))
        
    histo_dic["hq"] =  higher_quark
    histo_dic["hg"] =  higher_gluon
    histo_dic["ho"] =  higher_b_quark
    
    histo_dic["hmc"] =  higher_quark.Clone()
    histo_dic["hmc"].SetName(pt+"higher_all_jets"+var+"_"+name)
    histo_dic["hmc"].Add(higher_gluon)
    histo_dic["hmc"].Add(higher_b_quark)
    
    histo_dic["lq"] =  lower_quark
    histo_dic["lg"] =  lower_gluon
    histo_dic["lo"] =  lower_b_quark
    
    histo_dic["lmc"] =  lower_quark.Clone()
    histo_dic["lmc"].SetName(pt+"lower_all_jets"+var+"_"+name)
    histo_dic["lmc"].Add(lower_gluon)
    histo_dic["lmc"].Add(lower_b_quark)
    
    
    return(histo_dic)        






gStyle.SetOptStat(0)

for k in range(0,6):  #for only dijet event, start frTom jet pT>500 GeV

     for var_index,var in enumerate(var_array):
    
            min = bin[k]
            max = bin[k+1]
            for e in range(0,len(eta_bin)):
        
                    print(var,str(eta_bin[e]))
                    
                    nominal_jets =  ReadingDijetMCEta(fdijetMC,str(min),var,eta_bin[e],"mc")
                
                            
                    higher_quark = nominal_jets["hq"]
                    lower_quark = nominal_jets["lq"]
                    higher_gluon = nominal_jets["hg"]
                    lower_gluon = nominal_jets["lg"]
                    higher_other = nominal_jets["ho"]
                    lower_other = nominal_jets["lo"]        
                    higher_mc = nominal_jets["hmc"]
                    lower_mc = nominal_jets["lmc"]        

                    
                    higher_quark_unc = unc_array(higher_quark)
                    higher_gluon_unc = unc_array(higher_gluon)
                    lower_quark_unc = unc_array(lower_quark)
                    lower_gluon_unc = unc_array(lower_gluon)
                    higher_mc_unc = unc_array(higher_mc)
                    lower_mc_unc = unc_array(lower_mc)
        
                    
                    ToT_Fq2_unc = higher_quark_unc.sum()
                    ToT_Fg2_unc = higher_gluon_unc.sum()
            
                    ToT_Cq2_unc = lower_quark_unc.sum()
                    ToT_Cg2_unc = lower_gluon_unc.sum()
            
                    # calculate the fraction of forward(higher) / central(lower) quark or gluon jet
                    fg_unc=ToT_Fg2_unc/(ToT_Fg2_unc+ToT_Fq2_unc)
                    cg_unc=ToT_Cg2_unc/(ToT_Cq2_unc+ToT_Cg2_unc)
                    fq_unc=1.-fg_unc
                    cq_unc=1.-cg_unc
            
                    factor_quark_unc = lower_quark_unc
                    factor_gluon_unc = lower_gluon_unc
                    
                    
                    higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
                    higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
                    lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
                    lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()
        
                    higher_mc_unc = higher_mc_unc/higher_mc_unc.sum()
                    lower_mc_unc = lower_mc_unc/lower_mc_unc.sum()        

        
                        
                    higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
                    higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
                    lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
                    lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()
        
                    higher_mc_unc = higher_mc_unc/higher_mc_unc.sum()
                    lower_mc_unc = lower_mc_unc/lower_mc_unc.sum()   
                    
                    higher = higher_mc.Clone()
                    lower = lower_mc.Clone()
            
                    higher_unc = higher_mc_unc
                    lower_unc  = lower_mc_unc
            
                    
        
                    # kinematics
                    set_hist(higher_quark,higher_quark_unc)
                    set_hist(higher_gluon,higher_gluon_unc)
                    set_hist(lower_quark,lower_quark_unc)
                    set_hist(lower_gluon,lower_gluon_unc)   
                    
                    
                    higher_quark_hist = higher_quark.Clone()
                    higher_gluon_hist = higher_quark.Clone() 
                    lower_quark_hist = higher_quark.Clone()
                    lower_gluon_hist = higher_quark.Clone()
        
                    for i in range(1,higher_quark.GetNbinsX()+1):
                            if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                    factor_gluon_unc[i-1] = higher_gluon_unc[i-1]/lower_gluon_unc[i-1]
                                    factor_quark_unc[i-1] = higher_quark_unc[i-1]/lower_quark_unc[i-1]
                            else:
                                    factor_gluon_unc[i-1] = unc.ufloat(1, 0)
                                    factor_quark_unc[i-1] = unc.ufloat(1, 0)                    
                    for i in range(1,higher_quark.GetNbinsX()+1):
                        higher_quark_hist.SetBinContent(i,higher_quark_unc[i-1].nominal_value)
                        higher_quark_hist.SetBinError(i,higher_quark_unc[i-1].std_dev)
                        lower_quark_hist.SetBinContent(i,lower_quark_unc[i-1].nominal_value)
                        lower_quark_hist.SetBinError(i,lower_quark_unc[i-1].std_dev)
                        higher_gluon_hist.SetBinContent(i,higher_gluon_unc[i-1].nominal_value)
                        higher_gluon_hist.SetBinError(i,higher_gluon_unc[i-1].std_dev)
                        lower_gluon_hist.SetBinContent(i,lower_gluon_unc[i-1].nominal_value)
                        lower_gluon_hist.SetBinError(i,lower_gluon_unc[i-1].std_dev)
                        
                    factor_quark_hist = higher_quark.Clone()
                    factor_gluon_hist = higher_gluon.Clone()
                    for i in range(1,higher_quark.GetNbinsX()+1):
        
                            factor_quark_hist.SetBinContent(i,factor_quark_unc[i-1].nominal_value)
                            factor_quark_hist.SetBinError(i,factor_quark_unc[i-1].std_dev)
                            factor_gluon_hist.SetBinContent(i,factor_gluon_unc[i-1].nominal_value)
                            factor_gluon_hist.SetBinError(i,factor_gluon_unc[i-1].std_dev)
                    
                           
                    
                    c5 = TCanvas("","",500,500)
                    c5.Divide(2,1)
        
                    top = c5.cd(1)
                    top.SetPad(0.0,0.0,1.0,1.0)
                    top.SetFillColor(0)
                    top.SetBorderMode(0)
                    top.SetBorderSize(2)
                    top.SetTickx(1)
                    top.SetTicky(1)
                    top.SetLeftMargin(0.14)
                    top.SetRightMargin(0.055)
                    top.SetBottomMargin(0.3)#0.25
                    top.SetFrameBorderMode(0)
                    #top.SetLogy(1)
                    top.cd()
                    higher_quark_hist.SetMarkerColor(4)
                    higher_quark_hist.SetLineColor(4)
                    higher_quark_hist.SetMarkerSize(0.5)
                    higher_quark_hist.SetLineStyle(1)
                    higher_gluon_hist.SetMarkerColor(2)
                    higher_gluon_hist.SetLineColor(2)
                    higher_gluon_hist.SetMarkerSize(0.5)
                    higher_gluon_hist.SetLineStyle(1)         
                    lower_quark_hist.SetMarkerColor(4)
                    lower_quark_hist.SetLineColor(4)
                    lower_quark_hist.SetMarkerSize(0.5)
                    lower_quark_hist.SetLineStyle(2)
                    lower_gluon_hist.SetMarkerColor(2)
                    lower_gluon_hist.SetLineColor(2)
                    lower_gluon_hist.SetMarkerSize(0.5)
                    lower_gluon_hist.SetLineStyle(2)           
                    bot = c5.cd(2)
                    bot.SetPad(0.0,0.0,1.0,0.3)
                    bot.SetFillColor(0)
                    bot.SetBorderMode(0)
                    bot.SetBorderSize(2)
                    bot.SetTickx(1)
                    bot.SetTicky(1)
                    bot.SetLeftMargin(0.14)
                    bot.SetRightMargin(0.055)
                    bot.SetTopMargin(0.045)
                    bot.SetBottomMargin(0.4)
                    bot.SetFrameBorderMode(0)   
                    factor_quark_hist.SetMarkerColor(4)
                    factor_quark_hist.SetLineColor(4)
                    factor_quark_hist.SetMarkerSize(0.5)
                    factor_quark_hist.SetLineStyle(1)
                    factor_gluon_hist.SetMarkerColor(2)
                    factor_gluon_hist.SetLineColor(2)
                    factor_gluon_hist.SetMarkerSize(0.5)
                    factor_gluon_hist.SetLineStyle(1)   
                    #leg.AddEntry(gluon_data,"Extracted gluon (data)","p")
                    factor_quark_hist.SetMinimum(0)
                    factor_quark_hist.SetMaximum(2)
                    factor_quark_hist.GetYaxis().SetTitle("Forward/Central")        
                    factor_quark_hist.GetYaxis().SetRangeUser(0.7,1.3)
                    factor_quark_hist.GetXaxis().SetTitleOffset(1)
                    factor_quark_hist.GetXaxis().SetTitleSize(0.11)
                    factor_quark_hist.GetXaxis().SetLabelSize(0.1)
                    factor_quark_hist.GetXaxis().SetLabelOffset(0.03)
                    factor_quark_hist.GetYaxis().SetTitleSize(0.1)
                    factor_quark_hist.GetYaxis().SetTitleOffset(0.5)
                    factor_quark_hist.GetYaxis().SetLabelOffset(0.01)
                    factor_quark_hist.GetYaxis().SetLabelSize(0.1)
                    factor_quark_hist.GetYaxis().SetLabelOffset(0.02)
                    top.cd()
                    rmax = higher_gluon_hist.GetMaximum()*1.5
                    higher_quark_hist.GetYaxis().SetTitle("Normalized to unity")        
                    higher_quark_hist.SetMaximum(rmax)
                    higher_quark_hist.Draw("HIST")
                    higher_gluon_hist.Draw("HIST same")
                    lower_quark_hist.Draw("HIST same")
                    lower_gluon_hist.Draw("HIST same")

                    factor_quark_hist.GetXaxis().SetTitle(var_title_array[var_index])
                    leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  

        
                    leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  
                    leg.AddEntry(higher_quark_hist,"forward quark","l")
                    leg.AddEntry(lower_quark_hist,"central quark","l")     
                    leg.AddEntry(higher_gluon_hist,"forward gluon","l")
                    leg.AddEntry(lower_gluon_hist,"central gluon","l")  
                    leg.SetTextFont(42)
                    leg.SetFillColor(0)
                    leg.SetBorderSize(0)
                    leg.SetFillStyle(0)
                    leg.SetNColumns(1)
                    leg.Draw()        
                    myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
                    myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
                    myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
                    myText(0.18,0.70,"0<|\eta_{C}|<0.5,0.5<|\eta_{F}|<1")
                    bot.cd()
                    factor_quark_hist.Draw("HIST e")
                    factor_gluon_hist.Draw("HIST e same")
        
                    c5.Print(var+"_"+str(min)+"_"+mc+"_"+var+"_"+str(eta_bin[e])+"_distribution.png")           

bdt 0-0.5_0.5-1
eta 0-0.5_0.5-1
c1 0-0.5_0.5-1
width 0-0.5_0.5-1
ntrk 0-0.5_0.5-1
bdt 0-0.5_0.5-1
eta 0-0.5_0.5-1
c1 0-0.5_0.5-1
width 0-0.5_0.5-1
ntrk 0-0.5_0.5-1
bdt 0-0.5_0.5-1
eta 0-0.5_0.5-1
c1 0-0.5_0.5-1
width 0-0.5_0.5-1
ntrk 0-0.5_0.5-1
bdt 0-0.5_0.5-1
eta 0-0.5_0.5-1
c1 0-0.5_0.5-1
width 0-0.5_0.5-1
ntrk 0-0.5_0.5-1
bdt 0-0.5_0.5-1
eta 0-0.5_0.5-1
c1 0-0.5_0.5-1
width 0-0.5_0.5-1
ntrk 0-0.5_0.5-1
bdt 0-0.5_0.5-1
eta 0-0.5_0.5-1
c1 0-0.5_0.5-1
width 0-0.5_0.5-1
ntrk 0-0.5_0.5-1


Info in <TCanvas::Print>: png file bdt_500_pythia_bdt_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file eta_500_pythia_eta_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file c1_500_pythia_c1_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file width_500_pythia_width_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file ntrk_500_pythia_ntrk_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file bdt_600_pythia_bdt_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file eta_600_pythia_eta_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file c1_600_pythia_c1_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file width_600_pythia_width_0-0.5_0.5-1_distribution.png has been created
Info in <TCanvas::Print>: png file ntrk_600_pythia_ntrk_0-0.5_0.5-1_distribution.png has been 